In [1]:
import spacy

In [2]:
nlp=spacy.load("en_core_web_lg")

In [3]:
doc=nlp("dog cat banana house bhuijk")

In [4]:
for token in doc:
    print(token,":",token.has_vector)

dog : True
cat : True
banana : True
house : True
bhuijk : False


In [5]:
for token in doc:
    print(token,":",token.is_oov)

dog : False
cat : False
banana : False
house : False
bhuijk : True


In [6]:
base_token=nlp("bread")
base_token.vector.shape

(300,)

In [10]:
doc=nlp("bread sandwich burger car tiger human wheat")
for token in doc:
    print(f"{token.text}<->{base_token.text}:", token.similarity(base_token))

bread<->bread: 1.0
sandwich<->bread: 0.6874560014053445
burger<->bread: 0.5440373883702087
car<->bread: 0.1644114584391833
tiger<->bread: 0.14492353269643002
human<->bread: 0.21103659037655728
wheat<->bread: 0.6572456428272563


In [11]:
def similarity(base_word, words_to_compare):
    base_token=nlp(base_word)
    doc=nlp(words_to_compare)
    for token in doc:
        print(f"{token.text}<->{base_token.text}:",token.similarity(base_token))

In [18]:
similarity("iphone","apple samsung iphone dog BROKEN")

apple<->iphone: 0.6339781147910419
samsung<->iphone: 0.6678678014329177
iphone<->iphone: 1.0
dog<->iphone: 0.17431037640553934
BROKEN<->iphone: 0.2642890092754203


In [19]:
king=nlp.vocab["king"].vector
man=nlp.vocab["man"].vector
woman=nlp.vocab["woman"].vector
queen=nlp.vocab["queen"].vector
result=king-man+woman

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([result],[queen])

array([[0.7880844]], dtype=float32)

In [21]:
import pandas as pd

In [23]:
news_data=pd.read_csv(r"C:\Users\ggomare\Downloads\Fake_Real_Data.csv")

In [25]:
news_data

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real
...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake
9896,Trump consults Republican senators on Fed chie...,Real
9897,Trump lawyers say judge lacks jurisdiction for...,Real
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake


In [26]:
news_data["new_label"]=news_data["label"].apply(lambda x:1 if x=="Real" else 0)

In [27]:
news_data

,Text,label,new_label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1
...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,0
9896,Trump consults Republican senators on Fed chie...,Real,1
9897,Trump lawyers say judge lacks jurisdiction for...,Real,1
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,0


In [28]:
news_data["new_label"].value_counts()

0    5000
1    4900
Name: new_label, dtype: int64

In [29]:
news_data['vector']=news_data["Text"].apply(lambda x: nlp(x).vector)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x=news_data["vector"]
y=news_data["new_label"]
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=2022,stratify=news_data["new_label"])

In [33]:
import numpy as np
xtrain_2d=np.stack(xtrain)
xtest_2d=np.stack(xtest)

In [34]:
from sklearn.naive_bayes import MultinomialNB


In [36]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
new_xtrain=scaler.fit_transform(xtrain_2d)
new_xtest=scaler.fit_transform(xtest_2d)

In [39]:
clf=MultinomialNB()
clf.fit(new_xtrain,ytrain)
pred=clf.predict(new_xtest)

In [42]:
from sklearn.metrics import classification_report
print(classification_report(pred,ytest))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95      1057
           1       0.91      0.97      0.94       923

    accuracy                           0.94      1980
   macro avg       0.94      0.94      0.94      1980
weighted avg       0.94      0.94      0.94      1980



In [44]:
from sklearn.neighbors import KNeighborsClassifier

In [45]:
model=KNeighborsClassifier()

In [47]:
model.fit(new_xtrain,ytrain)
pred2=model.predict(new_xtest)
print(classification_report(pred2,ytest))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       983
           1       0.99      0.97      0.98       997

    accuracy                           0.98      1980
   macro avg       0.98      0.98      0.98      1980
weighted avg       0.98      0.98      0.98      1980



C:\Users\ggomare\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
